Projet Data Science : Création d'un indicateur de dangerosité des routes à destination des applications d'aide à la conduite 

Objectif du projet : Créer une échelle de dangerosité des routes et obtenir une carte sur laquelle on pourrait afficher la dangerosité des routes



3 datasets utilisés : 
- Accidents corporels géocodés de la circulation routière française de 2005 à 2020 : 
(licence ouverte : https://www.etalab.gouv.fr/wp-content/uploads/2014/05/Licence_Ouverte.pdf)

 https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/

- Base des unités urbaines françaises : description géométrique des agglomérations françaises 
(licence ouverte : https://www.etalab.gouv.fr/wp-content/uploads/2014/05/Licence_Ouverte.pdf)

https://www.insee.fr/fr/information/4802589

- Base de données géographiques : description géométrique des routes françaises
(licence :https://opendatacommons.org/licenses/odbl/)

https://www.openstreetmap.fr/


In [ ]:
#Importation des bibliothèques
pip install kmodes
from google.colab import drive
import numpy as np
import pandas as pd 
from kmodes.kmodes import KModes
import string
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### PARTIE DF

In [ ]:
#Importation tableau
data=pd.read_csv("/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/accidents.csv",sep=";",decimal=",")
df = pd.read_csv("/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/features_accidents_latest.csv",sep=";",decimal=",")
df = df.drop("Unnamed: 0", axis=1)
data

,cycleway,highway,lanes,lit,maxspeed,sidewalk,agg,int,bidirectionnel
0,False,residential,Unknown,False,50,False,True,False,True
1,False,unclassified,Unknown,False,Unknown,False,False,False,True
2,False,primary,2,False,90,False,False,True,True
3,False,residential,Unknown,False,50,False,True,True,True
4,False,tertiary,1,True,30,False,True,False,False
...,...,...,...,...,...,...,...,...,...
543080,False,unclassified,Unknown,False,Unknown,False,False,True,True
543081,False,primary,2,False,90,False,False,False,True
543082,False,secondary,2,False,90,False,False,False,True
543083,False,tertiary,2,False,90,False,False,False,True


In [ ]:
#Analyse des données

In [ ]:
df['maxspeed'].value_counts()/df.value_counts().sum()

50         0.484539
30         0.200328
Unknown    0.125384
90         0.090170
80         0.027896
70         0.027598
110        0.026405
130        0.017680
Name: maxspeed, dtype: float64

In [ ]:
df['sidewalk'].value_counts() / df['sidewalk'].value_counts().sum()

In [ ]:
#Représentation graphique
import plotly.express as px

px.bar(y=df['highway'].value_counts().values, x=list(df['highway'].value_counts().index))

In [ ]:
#CLustering
K = range(1,8)
cost_rectiligne = []
cost_idf = []
for num_clusters in list(K):
    kmode_rectiligne = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode_rectiligne.fit_predict(df2)
    cost_rectiligne.append(kmode.cost_)

    kmode_idf = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode_idf.fit_predict(df3)
    cost_idf.append(kmode.cost_)

    print("###########################",num_clusters)

In [ ]:
cost_idf

In [ ]:
#résultats du programme 
cost = [1695549.0,1404855.0,1231600.0,1123133.0,1053312.0,1004213.0,924802.0,903327.0,886110.0,836887.0,817480.0,786315.0,752753.0,747692.0,714137.0,684676.0,675572.0,672012.0,669904.0]

K = [i for i in range(1,len(cost)+1)]

#Etude du nombre de clusters

import plotly.express as px

fig=px.line(x=K,y=cost,title='Elbow Method For Optimal k',labels={'x':'Number of clusters','y':'Cost'})
fig.update_traces(mode='markers+lines')
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
kmode = KModes(n_clusters=7, init = "random", n_init = 5, verbose=1)
kmode.fit_predict(df)

In [ ]:

kmode = KModes(n_clusters=7, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(df)  


In [ ]:

from sklearn.metrics import silhouette_samples, silhouette_score
cluster_labels = clusterer.labels_ # Extract the labels = identifiant de ton cluster
silhouette_avg = silhouette_score(ruspini, cluster_labels) # Corresponding silhouette score = donne un score à ce nombre de cluster 
   
    
print(n, "Silhouette_score:", silhouette_avg)


In [ ]:
import string
##Determination de la dangerosité
def class_of_dangerosity(dictionnary,center):
  letters = list(string.ascii_lowercase)
  dangerosityClass = dict()
  new_center = dict()
  c = 0
  dangerosityList = list(dictionnary.values())
  while c < len(dangerosityList) :
    min = 100000000
    i_min = 0
    for i in range(len(dangerosityList)):
      if dangerosityList[i] < min :
        min = dangerosityList[i]
        i_min = i
    dangerosityList[i_min] = 100000000
    dangerosityClass[letters[c]] = [min,i_min]
    new_center[letters[c]] = center[i_min]
    c += 1 
    
  return dangerosityClass,new_center


#Fonction de dangerosité
def dangerosity(x,dictionnary,center):
  dangerosityDict, new_center= class_of_dangerosity(dictionnary,center)
  new_x = list()
  
  for e in x:
    for i in range(len(dangerosityDict)):
      if e == list(dangerosityDict.values())[i][1]:
        new_x.append(list(dangerosityDict)[i][0])

  return np.asarray(new_x), new_center


def func_center(center,df):
  dict_center=[]
  for i in range(len(center)):
    dict_inter = dict(zip(df.columns, list(center.values())[i]))
    dict_center.append(dict_inter)

  return dict(zip(list(center),np.asarray(dict_center)))


unique_c, counts_c = np.unique(clusters, return_counts=True)
label_of_dangerosity,center = dangerosity(clusters,dict(zip(unique_c,counts_c)),kmode.cluster_centroids_)
unique_d, counts_d = np.unique(label_of_dangerosity, return_counts=True)
center_of_dangerosity = func_center(center,df)
print('classes de dangerosités : ' ,dict(zip(unique_d,counts_d)))
print('centre des clusters : ', center_of_dangerosity)

In [ ]:
#Centre de dangerosité des clusters
df_centers = pd.DataFrame(index=list(center_of_dangerosity))
for e in df.columns :
  df_centers[e] = [list(center_of_dangerosity.values())[i][e] for i in range(len(center_of_dangerosity.values()))]
df_centers

In [ ]:
df_centers.to_csv("/content/drive/MyDrive/Dataset/cluster/cluster1.csv")

In [ ]:
#Centre des clusters
kmode.cluster_centroids_

df_centroid = pd.DataFrame(kmode.cluster_centroids_)
df_centroid.columns = df.columns
df_centroid

In [ ]:
#Etude graphique
import plotly.express as px
fig = px.bar(y=counts_d, x=unique_d)
fig.show()

In [ ]:
df_centers['class_of_dangerosity'] = list(center_of_dangerosity)
for i in range(len(list(center_of_dangerosity))):
  if df_centers.index[i]=='a' or df_centers.index[i]=='b' :
    df_centers['class_of_dangerosity'][i]='A'
  if df_centers.index[i]=='c' or df_centers.index[i]=='d' or df_centers.index[i]=='e':
    df_centers['class_of_dangerosity'][i]='B'
  if df_centers.index[i]=='f':
    df_centers['class_of_dangerosity'][i]='C'
  if df_centers.index[i]=='g':
    df_centers['class_of_dangerosity'][i]='D'

df_centers

In [ ]:
df_centers.to_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/cluster/cluster2.csv')

In [ ]:
def distance(rowA, rowB):
  return (rowA != rowB).sum()

def get_categorie(row, centroids):
  dist = [distance(row, features) for _, features in centroids.iterrows()]
  return min(range(len(dist)), key=dist.__getitem__)

In [ ]:
get_categorie(df_centroid.iloc[3], df_centroid)
#print(df_centroid.rows)

In [ ]:
df2 = df.sample(10000)

In [ ]:
categorie = df.apply(lambda row: get_categorie(row,df_centroid), axis=1)


In [ ]:
categorie.value_counts()

In [ ]:
fig = px.bar(y=categorie.value_counts().values, x=list(categorie.value_counts().index))
fig.show()

## PARTIE DF2

In [ ]:
#Importation données
df2 = pd.read_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/features_with_plan_france.csv',';')
df2 = df2.drop("Unnamed: 0", axis=1)

In [ ]:
df2['rectiligne'].value_counts()

In [ ]:
#Clustering
kmode2 = KModes(n_clusters=7, init = "random", n_init = 5, verbose=1)
clusters2 = kmode2.fit_predict(df2)  

In [ ]:
##determination of dangerosity
def class_of_dangerosity(dictionnary,center):
  letters = list(string.ascii_lowercase)
  dangerosityClass = dict()
  new_center = dict()
  c = 0
  dangerosityList = list(dictionnary.values())
  while c < len(dangerosityList) :
    min = 100000000
    i_min = 0
    for i in range(len(dangerosityList)):
      if dangerosityList[i] < min :
        min = dangerosityList[i]
        i_min = i
    dangerosityList[i_min] = 100000000
    dangerosityClass[letters[c]] = [min,i_min]
    new_center[letters[c]] = center[i_min]
    c += 1 
    
  return dangerosityClass,new_center



def dangerosity(x,dictionnary,center):
  dangerosityDict, new_center= class_of_dangerosity(dictionnary,center)
  new_x = list()
  
  for e in x:
    for i in range(len(dangerosityDict)):
      if e == list(dangerosityDict.values())[i][1]:
        new_x.append(list(dangerosityDict)[i][0])

  return np.asarray(new_x), new_center


def func_center(center,df):
  dict_center=[]
  for i in range(len(center)):
    dict_inter = dict(zip(df.columns, list(center.values())[i]))
    dict_center.append(dict_inter)

  return dict(zip(list(center),np.asarray(dict_center)))


unique_c2, counts_c2 = np.unique(clusters2, return_counts=True)
label_of_dangerosity2,center2 = dangerosity(clusters2,dict(zip(unique_c2,counts_c2)),kmode2.cluster_centroids_)
unique_d2, counts_d2 = np.unique(label_of_dangerosity2, return_counts=True)
center_of_dangerosity2 = func_center(center2,df2)
print('classes de dangerosités : ' ,dict(zip(unique_d2,counts_d2)))
print('centre des clusters : ', center_of_dangerosity2)

In [ ]:
#Représentation graphique
import plotly.express as px
fig = px.bar(y=counts_d2, x=unique_d2)
fig.show()

In [ ]:
df_centers2 = pd.DataFrame(index=list(center_of_dangerosity2))
for e in df2.columns :
  df_centers2[e] = [list(center_of_dangerosity2.values())[i][e] for i in range(len(center_of_dangerosity2.values()))]
df_centers2

In [ ]:
center_of_dangerosity2.values()

In [ ]:
df_centers2['class_of_dangerosity'] = list(center_of_dangerosity2)
for i in range(len(list(center_of_dangerosity2))):
  if df_centers2.index[i]=='a':
    df_centers2['class_of_dangerosity'][i]='A'
  if df_centers2.index[i]=='b' :
    df_centers2['class_of_dangerosity'][i]='B'
  if df_centers2.index[i]=='c' or df_centers2.index[i]=='d' or df_centers2.index[i]=='e':
    df_centers2['class_of_dangerosity'][i]='C'
  if df_centers2.index[i]=='f':
    df_centers2['class_of_dangerosity'][i]='D'
  if df_centers2.index[i]=='g':
    df_centers2['class_of_dangerosity'][i]='E'

df_centers2.to_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/cluster/cluster3.csv')

## PARTIE DF3

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/features_idf.csv',';')
df3 = df3.drop("Unnamed: 0", axis=1)
df3

In [ ]:
#CLustering
kmode3 = KModes(n_clusters=7, init = "random", n_init = 5, verbose=1)
clusters3 = kmode3.fit_predict(df3)  

In [ ]:
##determination of dangerosity
def class_of_dangerosity(dictionnary,center):
  letters = list(string.ascii_lowercase)
  dangerosityClass = dict()
  new_center = dict()
  c = 0
  dangerosityList = list(dictionnary.values())
  while c < len(dangerosityList) :
    min = 100000000
    i_min = 0
    for i in range(len(dangerosityList)):
      if dangerosityList[i] < min :
        min = dangerosityList[i]
        i_min = i
    dangerosityList[i_min] = 100000000
    dangerosityClass[letters[c]] = [min,i_min]
    new_center[letters[c]] = center[i_min]
    c += 1 
    
  return dangerosityClass,new_center



def dangerosity(x,dictionnary,center):
  dangerosityDict, new_center= class_of_dangerosity(dictionnary,center)
  new_x = list()
  
  for e in x:
    for i in range(len(dangerosityDict)):
      if e == list(dangerosityDict.values())[i][1]:
        new_x.append(list(dangerosityDict)[i][0])

  return np.asarray(new_x), new_center


def func_center(center,df):
  dict_center=[]
  for i in range(len(center)):
    dict_inter = dict(zip(df.columns, list(center.values())[i]))
    dict_center.append(dict_inter)

  return dict(zip(list(center),np.asarray(dict_center)))


unique_c3, counts_c3 = np.unique(clusters3, return_counts=True)
label_of_dangerosity3,center3 = dangerosity(clusters3,dict(zip(unique_c3,counts_c3)),kmode3.cluster_centroids_)
unique_d3, counts_d3 = np.unique(label_of_dangerosity3, return_counts=True)
center_of_dangerosity3 = func_center(center3,df3)
print('classes de dangerosités : ' ,dict(zip(unique_d3,counts_d3)))
print('centre des clusters : ', center_of_dangerosity3)

In [ ]:
import plotly.express as px
fig = px.bar(y=counts_d3, x=unique_d3)
fig.show()

In [ ]:
df_centers3 = pd.DataFrame(index=list(center_of_dangerosity3))
for e in df3.columns :
  df_centers3[e] = [list(center_of_dangerosity3.values())[i][e] for i in range(len(center_of_dangerosity3.values()))]
df_centers3

In [ ]:
df_centers3['class_of_dangerosity'] = list(center_of_dangerosity3)
for i in range(len(list(center_of_dangerosity3))):
  if df_centers3.index[i]=='a':
    df_centers3['class_of_dangerosity'][i]='A'
  if df_centers3.index[i]=='b' or df_centers3.index[i]=='c':
    df_centers3['class_of_dangerosity'][i]='B'
  if df_centers3.index[i]=='d' or df_centers3.index[i]=='e':
    df_centers3['class_of_dangerosity'][i]='C'
  if df_centers3.index[i]=='f':
    df_centers3['class_of_dangerosity'][i]='D'
  if df_centers3.index[i]=='g':
    df_centers3['class_of_dangerosity'][i]='E'

df_centers3

In [ ]:
df_centers3.to_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/cluster/cluster4.csv')

## PARTIE DF4

In [ ]:
df4 = pd.read_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/accidents_with_tags.csv',';')

#enlever les points hors france métropolitaine
from shapely.geometry import Point, Polygon
points = [[48.426284,-6.510319], [51.571607, 2.697183], [48.977333, 9.042430], [43.385888, 8.602449], [41.879378, 3.147302], [43.755225, -7.691633]]
france = Polygon(points)
france_metropolitaine = df4.apply(lambda row: france.contains(Point(float(row["latitude"]), float(row["longitude"]))), axis=1) 
df4= df4[france_metropolitaine]

df4 = df4[df4['way_tags'].notna()]
df4 = df4.drop(df4[df4['way_tags']==-1].index,axis=0)
df4 = df4.reset_index()

In [ ]:
def index_drop(df,max):
  index = list()
  dict_apparition = dict()
  for i in range(len(df)):
    dict_apparition[df['way_id'][i]]=0
  
  for i in range(len(df)):
    dict_apparition[df['way_id'][i]] += 1
    if dict_apparition[df['way_id'][i]]<=max:
      index.append(i)
  
  
  return index



In [ ]:
index = index_drop(df4,3)

df = pd.read_csv("/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/features_accidents_latest.csv",sep=";",decimal=",")
df = df.drop("Unnamed: 0", axis=1)
df4 = df.loc[index]
df4

In [ ]:
#CLustering
kmode4 = KModes(n_clusters=7, init = "random", n_init = 5, verbose=1)
clusters4 = kmode4.fit_predict(df4)  

In [ ]:
def class_of_dangerosity(dictionnary,center):
  letters = list(string.ascii_lowercase)
  dangerosityClass = dict()
  new_center = dict()
  c = 0
  dangerosityList = list(dictionnary.values())
  while c < len(dangerosityList) :
    min = 100000000
    i_min = 0
    for i in range(len(dangerosityList)):
      if dangerosityList[i] < min :
        min = dangerosityList[i]
        i_min = i
    dangerosityList[i_min] = 100000000
    dangerosityClass[letters[c]] = [min,i_min]
    new_center[letters[c]] = center[i_min]
    c += 1 
    
  return dangerosityClass,new_center



def dangerosity(x,dictionnary,center):
  dangerosityDict, new_center= class_of_dangerosity(dictionnary,center)
  new_x = list()
  
  for e in x:
    for i in range(len(dangerosityDict)):
      if e == list(dangerosityDict.values())[i][1]:
        new_x.append(list(dangerosityDict)[i][0])

  return np.asarray(new_x), new_center


def func_center(center,df):
  dict_center=[]
  for i in range(len(center)):
    dict_inter = dict(zip(df.columns, list(center.values())[i]))
    dict_center.append(dict_inter)

  return dict(zip(list(center),np.asarray(dict_center)))


unique_c4, counts_c4 = np.unique(clusters4, return_counts=True)
label_of_dangerosity4,center4 = dangerosity(clusters4,dict(zip(unique_c4,counts_c4)),kmode4.cluster_centroids_)
unique_d4, counts_d4 = np.unique(label_of_dangerosity4, return_counts=True)
center_of_dangerosity4 = func_center(center4,df4)
print('classes de dangerosités : ' ,dict(zip(unique_d4,counts_d4)))
print('centre des clusters : ', center_of_dangerosity4)

In [ ]:
import plotly.express as px
fig = px.bar(y=counts_d4, x=unique_d4)
fig.show()

In [ ]:
df_centers4 = pd.DataFrame(index=list(center_of_dangerosity4))
for e in df4.columns :
  df_centers4[e] = [list(center_of_dangerosity4.values())[i][e] for i in range(len(center_of_dangerosity4.values()))]
df_centers4

In [ ]:
df_centers4['class_of_dangerosity'] = list(center_of_dangerosity4)
for i in range(len(list(center_of_dangerosity4))):
  if df_centers4.index[i]=='a'or df_centers4.index[i]=='b':
    df_centers4['class_of_dangerosity'][i]='A'
  if df_centers4.index[i]=='c':
    df_centers4['class_of_dangerosity'][i]='B'
  if df_centers4.index[i]=='d':
    df_centers4['class_of_dangerosity'][i]='C'
  if df_centers4.index[i]=='e':
    df_centers4['class_of_dangerosity'][i]='D'
  if df_centers4.index[i]=='f':
    df_centers4['class_of_dangerosity'][i]='E'
  if df_centers4.index[i]=='g':
    df_centers4['class_of_dangerosity'][i]='F'

df_centers4





In [ ]:
df_centers4.to_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/cluster/cluster5.csv')

## PARTIE DF5

In [ ]:
df5 = pd.read_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/accidents_with_tags.csv',';')

#enlever les points hors france métropolitaine
from shapely.geometry import Point, Polygon
points = [[48.426284,-6.510319], [51.571607, 2.697183], [48.977333, 9.042430], [43.385888, 8.602449], [41.879378, 3.147302], [43.755225, -7.691633]]
france = Polygon(points)
france_metropolitaine = df5.apply(lambda row: france.contains(Point(float(row["latitude"]), float(row["longitude"]))), axis=1) 
df5= df5[france_metropolitaine]

df5 = df5[df5['way_tags'].notna()]
df5 = df5.drop(df5[df5['way_tags']==-1].index,axis=0)
df5 = df5.reset_index()

In [ ]:
def index_drop(df,max):
  index = list()
  dict_apparition = dict()
  for i in range(len(df)):
    dict_apparition[df['way_id'][i]]=0
  
  for i in range(len(df)):
    dict_apparition[df['way_id'][i]] += 1
    if dict_apparition[df['way_id'][i]]<=max:
      index.append(i)
  
  
  return index


In [ ]:
index = index_drop(df5,3)

df2 = pd.read_csv("/content/drive/MyDrive/Projet_Data_Science/Dataset/Accidents/features_with_plan_france.csv",sep=";",decimal=",")
df2 = df2.drop("Unnamed: 0", axis=1)
df5 = df2.loc[index]
df5

In [ ]:
kmode5 = KModes(n_clusters=7, init = "random", n_init = 5, verbose=1)
clusters5 = kmode5.fit_predict(df5) 

In [ ]:
def class_of_dangerosity(dictionnary,center):
  letters = list(string.ascii_lowercase)
  dangerosityClass = dict()
  new_center = dict()
  c = 0
  dangerosityList = list(dictionnary.values())
  while c < len(dangerosityList) :
    min = 100000000
    i_min = 0
    for i in range(len(dangerosityList)):
      if dangerosityList[i] < min :
        min = dangerosityList[i]
        i_min = i
    dangerosityList[i_min] = 100000000
    dangerosityClass[letters[c]] = [min,i_min]
    new_center[letters[c]] = center[i_min]
    c += 1 
    
  return dangerosityClass,new_center



def dangerosity(x,dictionnary,center):
  dangerosityDict, new_center= class_of_dangerosity(dictionnary,center)
  new_x = list()
  
  for e in x:
    for i in range(len(dangerosityDict)):
      if e == list(dangerosityDict.values())[i][1]:
        new_x.append(list(dangerosityDict)[i][0])

  return np.asarray(new_x), new_center


def func_center(center,df):
  dict_center=[]
  for i in range(len(center)):
    dict_inter = dict(zip(df.columns, list(center.values())[i]))
    dict_center.append(dict_inter)

  return dict(zip(list(center),np.asarray(dict_center)))


unique_c5, counts_c5 = np.unique(clusters5, return_counts=True)
label_of_dangerosity5,center5 = dangerosity(clusters5,dict(zip(unique_c5,counts_c5)),kmode5.cluster_centroids_)
unique_d5, counts_d5 = np.unique(label_of_dangerosity5, return_counts=True)
center_of_dangerosity5 = func_center(center5,df5)
print('classes de dangerosités : ' ,dict(zip(unique_d5,counts_d5)))
print('centre des clusters : ', center_of_dangerosity5)

In [ ]:
import plotly.express as px
fig = px.bar(y=counts_d5, x=unique_d5)
fig.show()

In [ ]:
df_centers5 = pd.DataFrame(index=list(center_of_dangerosity5))
for e in df5.columns :
  df_centers5[e] = [list(center_of_dangerosity5.values())[i][e] for i in range(len(center_of_dangerosity5.values()))]
df_centers5

In [ ]:

df_centers5['class_of_dangerosity'] = list(center_of_dangerosity5)
for i in range(len(list(center_of_dangerosity5))):
  if df_centers5.index[i]=='a':
    df_centers5['class_of_dangerosity'][i]='A'
  if df_centers5.index[i]=='b':
    df_centers5['class_of_dangerosity'][i]='B'
  if df_centers5.index[i]=='c':
    df_centers5['class_of_dangerosity'][i]='C'
  if df_centers5.index[i]=='d':
    df_centers5['class_of_dangerosity'][i]='D'
  if df_centers5.index[i]=='e':
    df_centers5['class_of_dangerosity'][i]='E'
  if df_centers5.index[i]=='f' or df_centers5.index[i]=='g':
    df_centers5['class_of_dangerosity'][i]='F'

df_centers5

In [ ]:
df_centers5.to_csv('/content/drive/MyDrive/Projet_Data_Science/Dataset/cluster/cluster6.csv')